In [155]:
import pandas as pd
import numpy as np

In [156]:
weeks = pd.read_csv('./data2021/games.csv')['week'].max()

In [157]:
player_stats = pd.read_csv("merged_df.csv").drop('Unnamed: 0', axis='columns')

In [158]:
play_data = pd.read_csv("./data2021/plays.csv")

In [159]:
player_stats.head()

,Acceleration,Age,Agility,Awareness,BC Vision,Block Shedding,Break Sack,Break Tackle,Carrying,Catch In Traffic,...,displayName,height,nflId,nopunctuationname_x,nopunctuationname_y,position,updatedName,updatedName2_x,updatedName2_y,weight
0,93,27,97,90,74,59,13,63,54,50,...,Desmond Trufant,72,2539334,Desmond Trufant,Desmond Trufant,CB,Desmond Trufant,Desmond Trufant,Desmond Trufant,190
1,87,27,71,77,70,33,12,33,74,74,...,Desmond Trufant,72,2539334,Desmond Trufant,Tommy Bohanon,CB,Desmond Trufant,Desmond Trufant,Tommy Bohanon,190
2,93,29,94,82,79,57,15,65,64,45,...,Robert Alford,70,2539653,Robert Alford,Robert Alford,CB,Robert Alford,Robert Alford,Robert Alford,186
3,89,26,92,87,82,60,15,75,65,60,...,Ricardo Allen,69,2543850,Ricardo Allen,Ricardo Allen,SS,Ricardo Allen,Ricardo Allen,Ricardo Allen,186
4,90,26,92,72,78,35,23,77,68,85,...,Ricardo Allen,69,2543850,Ricardo Allen,Justin Hardy,SS,Ricardo Allen,Ricardo Allen,Justin Hardy,186


In [160]:
player_stats.Team.unique()

array(['Falcons', 'Jaguars', 'Raiders', 'Packers', 'Seahawks', 'Bears',
       'Cowboys', 'Broncos', 'Patriots', 'Redskins', 'Eagles', 'Saints',
       'Colts', 'Cardinals', 'Bengals', 'Steelers', 'Jets', 'Vikings',
       'Panthers', 'Ravens', 'Bills', '49ers', 'Lions', 'Dolphins',
       'Texans', 'Titans', 'Giants', 'Chargers', 'Browns', 'Buccaneers',
       'Chiefs', 'Rams'], dtype=object)

In [161]:
# Get name to match play_data format
player_stats["abrev_name"] = player_stats.displayName.apply(lambda x: str(x)[0]+ "."+"".join(str(x).split(" ")[1:]))
player_stats.abrev_name = player_stats.abrev_name.apply(lambda x: str(x).lower())

In [162]:
player_stats.abrev_name

0       d.trufant
1       d.trufant
2        r.alford
3         r.allen
4         r.allen
          ...    
1861     j.reaves
1862        t.way
1863      m.smith
1864      m.smith
1865    s.mannion
Name: abrev_name, Length: 1866, dtype: object

In [163]:
team_mapping = {"PHI": "Eagles", "BUF": "Bills", "ATL": "Falcons", "PIT": "Steelers", "CLE": "Browns", "CIN": "Bengals", "IND": "Colts", "TEN": "Titans", "MIA": "Dolphins", "BAL": "Ravens", "NE": "Patriots", "HOU": "Texans", "JAX": "Jaguars", "NYG": "Giants", "NO": "Saints", "TB": "Buccaneers", "WAS": "Commanders", "ARI": "Cardinals", "CAR": "Panthers", "DAL": "Cowboys", "GB": "Packers", "CHI": "Bears", "NYJ": "Jets", "DET": "Lions", "OAK": "Raiders", "LA": "Rams", "MIN": "Vikings", "LAC": "Chargers", "KC": "Chiefs", "SF": "49ers", "DEN": "Broncos", "SEA": "Seahawks", "LV": "Raiders"}

In [164]:
play_data['Team'] = play_data.possessionTeam.apply(lambda x: team_mapping[str(x)])

In [165]:
play_data.possessionTeam.unique()

array(['ATL', 'PHI', 'PIT', 'CLE', 'CIN', 'IND', 'TEN', 'MIA', 'BAL',
       'BUF', 'NE', 'HOU', 'JAX', 'NYG', 'NO', 'TB', 'WAS', 'ARI', 'CAR',
       'DAL', 'GB', 'CHI', 'NYJ', 'DET', 'OAK', 'LA', 'MIN', 'LAC', 'KC',
       'SF', 'DEN', 'SEA'], dtype=object)

In [166]:
# Remove time from play description
playDescText = play_data.playDescription.apply(lambda x: " ".join(str(x).split()[1:]))
# Remove formation from play description
playDescText = playDescText.apply(lambda x: "".join(x.split(")")[1:]) if str(x).startswith("(") else x)
playDescText

0        M.Ryan pass short right to J.Jones pushed ob a...
1        M.Ryan pass incomplete short right to C.Ridley...
2          M.Ryan pass incomplete short left to D.Freeman.
3         M.Ryan pass deep left to J.Jones to PHI 6 for...
4         M.Ryan pass incomplete short right to D.Freeman.
                               ...                        
19234    J.Johnson pass incomplete short left [K.Correa...
19235     B.Gabbert pass incomplete short left to T.Tay...
19236     P.Rivers pass deep right to K.Allen to BLT 24...
19237    P.Rivers pass incomplete deep middle to K.Alle...
19238     L.Jackson pass incomplete short left to M.Cra...
Name: playDescription, Length: 19239, dtype: object

In [167]:
# People who received the ball
receivers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("to")+1] if ("pass" in str(x).lower().split(" ") and "to" in str(x).lower().split(" ")) & ("intercept" not in str(x).lower()) else np.nan)
# Remove full stop from some names at end
receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."] = receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."].apply(lambda x: "".join(str(x)[:-1]))
# 2 values have NO # instead of names, so we can remove these
receivers.loc[receivers=="no"] = np.nan

In [168]:
play_data['abrev_name'] = receivers # column representing who received
receiver_stats = play_data.merge(player_stats, how='left', left_on=['Team','abrev_name'], right_on=['Team','abrev_name'])
receiver_stats = receiver_stats.rename(columns={'abrev_name': "receiver"})

In [169]:
receiver_stats.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,displayName,height,nflId,nopunctuationname_x,nopunctuationname_y,position,updatedName,updatedName2_x,updatedName2_y,weight
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,Josh Jones,74,2558119.0,Josh Jones,Mackendy Cheridor,SS,Josh Jones,Josh Jones,Mackendy Cheridor,220.0
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,Calvin Ridley,73,2560854.0,Calvin Ridley,Calvin Ridley,WR,Calvin Ridley,Calvin Ridley,Calvin Ridley,190.0
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,Devonta Freeman,68,2543583.0,Devonta Freeman,Devonta Freeman,RB,Devonta Freeman,Devonta Freeman,Devonta Freeman,206.0
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,Josh Jones,74,2558119.0,Josh Jones,Mackendy Cheridor,SS,Josh Jones,Josh Jones,Mackendy Cheridor,220.0
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,Devonta Freeman,68,2543583.0,Devonta Freeman,Devonta Freeman,RB,Devonta Freeman,Devonta Freeman,Devonta Freeman,206.0


In [170]:
#Drop non-Madden players and anomalies
receiver_stats.dropna(axis='rows', subset='nflId', inplace=True)
receiver_stats.reset_index(drop=True, inplace=True)

In [171]:
len(receiver_stats['nflId'].unique())

348

In [172]:
feature_columns = ['Age', 'Height', 'Weight', 'Overall', 'Speed',
 'Acceleration', 'Agility', 'Change of Dir', 'Strength', 'Jumping',
 'Awareness', 'Carrying', 'Break Tackle', 'Juke Move', 'Spin Move',
 'Trucking', 'Stiff Arm', 'BC Vision', 'Catching', 'Catch In Traffic',
 'Spec Catch', 'Release', 'Short RR', 'Medium RR', 'Deep RR',
 'Throw Power', 'Throw Acc Short', 'Throw Acc Mid', 'Throw Acc Deep',
 'Throw Under Pressure', 'Throw On The Run', 'Play Action', 'Break Sack',
 'Run Block', 'Run Block Power', 'Run Block Finesse', 'Pass Block',
 'Pass Block Power', 'Pass Block Finesse', 'Impact Blocking',
 'Lead Blocking', 'Tackle', 'Hit Power', 'Pursuit', 'Man Coverage',
 'Zone Coverage', 'Press', 'Play Recognition', 'Power Moves',
 'Finesse Moves', 'Block Shedding', 'Kick Power', 'Kick Accuracy',
 'Kick Return', 'Stamina', 'Injury', 'Toughness', 'Years Pro']

In [173]:
total_yards_by_stats = pd.DataFrame({'nflId': receiver_stats['nflId'].astype('int32').unique()})

In [174]:
total_yards_by_stats['total_yards'] = total_yards_by_stats['nflId'].map(lambda x: receiver_stats[receiver_stats['nflId'] == x]['offensePlayResult'].sum() / weeks)

In [175]:
total_yards_by_stats = total_yards_by_stats.merge(player_stats[feature_columns + ['nflId']], how='inner', on='nflId')
total_yards_by_stats

,nflId,total_yards,Age,Height,Weight,Overall,Speed,Acceleration,Agility,Change of Dir,...,Power Moves,Finesse Moves,Block Shedding,Kick Power,Kick Accuracy,Kick Return,Stamina,Injury,Toughness,Years Pro
0,2558119,97.352941,23,74,220,74,92,91,88,67,...,41,54,44,25,19,48,93,88,82,1
1,2558119,97.352941,23,74,215,63,92,93,85,63,...,32,36,49,18,17,45,87,85,81,0
2,2558119,97.352941,23,77,275,63,76,85,73,34,...,67,70,69,18,14,10,85,86,80,0
3,2558119,97.352941,23,82,310,62,58,76,59,24,...,24,18,29,19,19,10,83,94,97,1
4,2558119,97.352941,23,77,290,60,67,64,58,10,...,10,10,20,12,11,10,84,83,82,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,2556273,0.000000,24,72,221,59,92,91,81,65,...,10,10,29,28,22,72,86,89,87,2
575,310,0.294118,33,76,217,89,72,81,69,56,...,10,10,25,25,25,10,92,98,94,10
576,2558875,0.294118,23,76,232,65,81,87,85,58,...,22,16,26,22,20,10,85,86,84,1
577,2558864,0.000000,24,79,316,72,66,79,64,15,...,10,10,15,24,22,10,76,90,67,3


In [194]:
player_stats[player_stats['nflId'] == 2558864]

,Acceleration,Age,Agility,Awareness,BC Vision,Block Shedding,Break Sack,Break Tackle,Carrying,Catch In Traffic,...,height,nflId,nopunctuationname_x,nopunctuationname_y,position,updatedName,updatedName2_x,updatedName2_y,weight,abrev_name
1558,79,24,64,85,10,15,12,9,40,10,...,77,2558864,Mason Schreck,Andrus Peat,TE,Mason Schreck,Mason Schreck,Andrus Peat,252,m.schreck
1559,89,24,77,52,75,36,18,48,76,75,...,77,2558864,Mason Schreck,Mason Schreck,TE,Mason Schreck,Mason Schreck,Mason Schreck,252,m.schreck


In [176]:
total_yards_by_stats.describe()

,nflId,total_yards,Age,Height,Weight,Overall,Speed,Acceleration,Agility,Change of Dir,...,Power Moves,Finesse Moves,Block Shedding,Kick Power,Kick Accuracy,Kick Return,Stamina,Injury,Toughness,Years Pro
count,5.790000e+02,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,...,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000
mean,2.461483e+06,17.564056,25.374784,73.675302,230.991364,74.143351,83.704663,87.519862,82.246978,59.010363,...,23.854922,23.918826,40.568221,22.018998,20.210708,41.569948,87.518135,87.590674,81.328152,3.015544
std,4.435808e+05,20.861214,2.952822,2.846401,38.041658,8.517548,8.213047,4.970244,9.827903,22.778607,...,21.704520,21.154449,17.760228,10.551963,9.860646,28.950498,4.774601,3.986133,7.444565,2.871185
min,3.100000e+02,-0.647059,21.000000,66.000000,160.000000,39.000000,55.000000,64.000000,47.000000,10.000000,...,5.000000,5.000000,9.000000,10.000000,10.000000,5.000000,69.000000,69.000000,50.000000,0.000000
25%,2.543457e+06,2.352941,23.000000,72.000000,203.500000,68.000000,80.000000,86.000000,76.500000,44.000000,...,10.000000,10.000000,27.000000,17.500000,15.000000,10.000000,85.000000,86.000000,77.000000,1.000000
50%,2.555220e+06,9.647059,25.000000,74.000000,221.000000,73.000000,86.000000,89.000000,85.000000,65.000000,...,10.000000,10.000000,35.000000,20.000000,19.000000,40.000000,88.000000,88.000000,82.000000,2.000000
75%,2.558270e+06,25.264706,27.000000,76.000000,253.000000,80.000000,90.000000,91.000000,89.000000,76.000000,...,30.000000,33.000000,49.000000,24.500000,22.000000,70.500000,91.000000,90.000000,86.000000,4.000000
max,2.561482e+06,97.352941,41.000000,82.000000,355.000000,99.000000,98.000000,97.000000,98.000000,95.000000,...,93.000000,95.000000,96.000000,96.000000,89.000000,98.000000,99.000000,98.000000,99.000000,18.000000


In [177]:
X = total_yards_by_stats[feature_columns]
X.shape

(579, 58)

In [178]:
Y = total_yards_by_stats['total_yards']
Y.shape

(579,)

In [179]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=0)

In [180]:
select_k_best=False

In [181]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

if select_k_best:
    k_best = SelectKBest(mutual_info_regression, k=20).fit(X_train, Y_train)
    X_train = k_best.transform(X_train)
    X_test = k_best.transform(X_test)
    print(k_best.get_feature_names_out())

In [182]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.07785811254167374

In [183]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,116.000000
mean,16.364857
std,10.678950
min,-18.486427
25%,9.698108
50%,16.262616
75%,23.114650
max,41.653666


In [184]:
from sklearn import svm
reg = svm.SVR().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-0.01132904014129732

In [185]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,116.000000
mean,9.768642
std,2.494649
min,3.329923
25%,8.501616
50%,9.805984
75%,10.794575
max,15.730600


In [186]:
from sklearn import tree
reg = tree.DecisionTreeRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-1.0894766430087746

In [187]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,116.000000
mean,19.666836
std,24.012061
min,-0.647059
25%,3.750000
50%,10.058824
75%,24.573529
max,97.352941


In [188]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.10990190564008873

In [189]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,116.000000
mean,17.784554
std,11.867597
min,5.297020
25%,9.651441
50%,13.901353
75%,20.234375
max,58.612598


In [190]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(learning_rate_init = 0.01, learning_rate='adaptive').fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.1456491538404917

In [191]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,116.000000
mean,15.743952
std,10.248800
min,-2.480691
25%,8.161416
50%,13.737968
75%,22.987919
max,45.518381
